<a href="https://colab.research.google.com/github/mairihallman/DUDADS-Chapters/blob/main/dl/dl-notebooks/introductory-ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductory Example - Linear Regression with Pytorch and Lighting

This example will help you get comfortable with Pytorch by performing linear regression with a neural network.

In [1]:
# install Lighting
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 36.0 MB/s eta 0:00:00


In [3]:
# required imports
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [4]:
set seed for reproducibility
torch.manual_seed(0)

We generate a dummy dataset for the equation y = 3x + 1.

In [12]:
x = torch.randn(100, 1) * 10 # sample x from a standard normal distribution multiplied by 10
y = 3 * x + 1 + torch.randn(100, 1) # compute y, add noise
dataset = TensorDataset(x, y) # initialize tensor dataset
train_loader = DataLoader(dataset, batch_size=10, shuffle=True) # initialize data loader

Now let's set up our model.

In [6]:
# define a new class for your model
class LinearRegressionModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    # forward pass
    def forward(self, x):
        return self.linear(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = nn.functional.mse_loss(y_pred, y)
        self.log('train_loss', loss)
        return loss

    # stochastic gradient descent for optimization
    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=0.01)

Now the fun part - training!

In [13]:
model = LinearRegressionModel()
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, train_loader)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 2      | train
------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name   | Type   | Params | Mode 
------------------------------------------
0 | linear | Linear | 2      | train
------------------------------------------
2         Trainable params
0         Non-trainable params
2    

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Let's see how our model did.

In [14]:
slope, intercept = model.linear.weight.item(), model.linear.bias.item()
print(f'Trained Model Parameters: Slope: {slope:.4f}, Intercept: {intercept:.4f}')

Trained Model Parameters: Slope: 2.9757, Intercept: 0.9874


Not bad!